## ML Model
### Constructing, training and Evaluation of at least three different models

### Cunstruction:
- Pipeline of multiple different ML-models

### Training:

- Training the different models with the splitted training dataset. Combining different combinations of features. USe vizualisation to see correlence between feature and goals scored in a match.

    - Models:
    - Multiple Linear Regression
    - Logistic Regression
    - KNN : KNearestNeighbours
        - Multiple k's
    - DecisionTreeRegressor
    - RandomForestRegressor
    - DecisionTreeClassifier
    - RandomForestClassifier
        - Different estimators (Ex n_estimators = 500)
        - mse = log_loss(y_val, pred)
    - MLPClassifier
    
#### Evaluation:
- Evaluate the different trained models with the validation dataset, check for over-/underfit:
    Methods:
    - MSE
        - Lower == Better
    - Delta
        - Lower == Lower MSE will give lower test_rmse hence lower delta better

In [1]:
from sklearn.linear_model import LinearRegression             as LR
from sklearn.linear_model import LogisticRegression           as LogReg

from sklearn.tree import DecisionTreeRegressor                as DTR
from sklearn.ensemble import RandomForestRegressor            as RFR
from sklearn.neighbors import KNeighborsRegressor             as KNN
from sklearn.tree import DecisionTreeClassifier               as DTC
from sklearn.ensemble import RandomForestClassifier           as RFC
from sklearn.neural_network import MLPClassifier              as MLPC

from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt 

import pandas as pd
import numpy as np

from matplotlib.pyplot import figure

import math
import random

from itertools import product
from itertools import combinations
import pickle

from sklearn.compose import ColumnTransformer
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, MaxAbsScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

from sklearn.feature_selection import SelectFromModel
import plotly.figure_factory as ff
plt.style.use('fivethirtyeight')
pd.set_option('display.max_rows', 1000)

In [2]:
numeric_features = ['HomeGround', 'AvgGoalsScored_90', 'GlsRatio', '%FPla', 'Avg_Age', 'OppGlsAgst90', 'tablePos', 'points']
## Reading the data created in the preparation stored in two CSV files
games = pd.read_csv("training_data_games.csv")
test_games = pd.read_csv("test_data_games.csv")

X = games[numeric_features]
Y = games['Goals']
X_train, X_valtest, Y_train, y_valtest = train_test_split(X, Y, test_size=0.3, random_state=69)
X_val, X_test, Y_val, Y_test = train_test_split(X_valtest, y_valtest, test_size=0.5, random_state=69)


## Creating a dummy-model, where the predicted score is a random number from a normal distribution from 0 - 6 (Max and min values of train-data). By doing this 30 times and taking the average of the RMSEs of the rounds, we get a more trustworthy RMSE to use for later evaluation:

In [3]:
def do_random_normal_predictions():
    random_predictions = []
    for i in range(len(X_test)):
        random_predictions.append(random.uniform(0, max(Y_train)))

    random_prediction_RMSE = mean_squared_error(
        Y_test, 
        random_predictions, 
        squared=False
    )
    return random_prediction_RMSE
normal_list = [do_random_normal_predictions() for x in range(30)]
avg_random_prediction_RMSE = np.average(normal_list)
print(f"RMSE from random normal-distribution: {avg_random_prediction_RMSE}")


RMSE from random normal-distribution: 2.569070225857679


## Creating a second dummy-model with the average amount of goals from the test-dataset, and comparing this with the actual Y_test-values:

In [4]:
avg_of_train_data = round(np.average(Y_train), 4)
print(f"Average goals scored from train-data: {avg_of_train_data}")
avg_predictions = [avg_of_train_data for i in range(len(Y_val))]
average_prediction_RMSE = mean_squared_error(
    Y_val, 
    avg_predictions, 
    squared=False
)
print(f"RMSE from average prediction: {average_prediction_RMSE}")

Average goals scored from train-data: 1.5591
RMSE from average prediction: 1.448303424700777


In [5]:
# imputers and different models
imputers = {
    'median': SimpleImputer(strategy='median'),
    'mean': SimpleImputer(strategy='mean'),
    'most_frequent' : SimpleImputer(strategy='most_frequent')
}
models = {'LinearRegression' : LR(positive=True),
          'LogisticRegression' : LogReg(),
          'DecisionTreeRegressor' : DTR(),
          'RandomForestRegressor_n100' : RFR(n_estimators = 100),
          'RandomForestRegressor_n200' : RFR(n_estimators = 200),
          'RandomForestRegressor_n300' : RFR(n_estimators = 300),
          'RandomForestRegressor_n400' : RFR(n_estimators = 400),
          'RandomForestRegressor_n500' : RFR(n_estimators = 500),
          'KNeighborsRegressor_03n' : KNN(n_neighbors=3),
          'KNeighborsRegressor_05n' : KNN(n_neighbors=5),
          'KNeighborsRegressor_07n' : KNN(n_neighbors=7),
          'KNeighborsRegressor_10n' : KNN(n_neighbors=10),
          'KNeighborsRegressor_13n' : KNN(n_neighbors=13),
          'DecisionTreeClassifier' : DTC(), 
          'RandomForestClassifier_250' : RFC(n_estimators=250),
          'RandomForestClassifier_500' : RFC(n_estimators=500),
          'RandomForestClassifier_750' : RFC(n_estimators=750),
          'RandomForestClassifier_1000' : RFC(n_estimators=1000),
             }
scalers = {
    'standard' : StandardScaler(copy = True, with_mean = True, with_std = True),
    'MinMax'   : MinMaxScaler(),
    'MaxAbs'   : MaxAbsScaler()
}
all_combinations = []
## Creating all combinations of imputers, models and scalers to find the optimal combination
for imp in imputers.items():
    for model in models.items():
        for scaler in scalers.items():
            this_set = [imp, model, scaler]
            all_combinations.append(this_set)
            


numeric_features = ['HomeGround', 'AvgGoalsScored_90', 'GlsRatio', '%FPla', 'Avg_Age', 'OppGlsAgst90', 'tablePos', 'points']



In [6]:
pipes = {imputer_key + '_' + model_key + "_" + scaler_key: Pipeline(
        steps=[
            ('preprocess', ColumnTransformer(
                transformers=[
                    ('num', Pipeline(
                        steps = [
                            ('impute', imputer), 
                            ('scaler', StandardScaler())
                        ]),
                     numeric_features)
                ]
            )
            ),
            ('regress', model)
        ]
    )
             for (imputer_key, imputer), (model_key, model), (scaler_key, scaler) in all_combinations}
    

In [15]:
validation_rmse = pd.DataFrame(
    {key: mean_squared_error(
        Y_val, 
        pipe.fit(X_train, Y_train).predict(X_val), 
        squared=False
    )
     for key, pipe in pipes.items()}.items(),
    columns=['model', 'rmse']
)
print("DONE")


DONE


In [16]:
models_w_atributes = []
models_test_rmse = []
models_delta = []
for model in validation_rmse['model']:
    
    ## Get the model to get its atributes
    mod = (pipes[model].steps)[-1][1]
    models_w_atributes.append(mod)
    
    m = pipes[model]
    test_rmse = mean_squared_error(Y_test, m.predict(X_test), squared=False)
    models_test_rmse.append(test_rmse)
    rmse_val = (validation_rmse.loc[validation_rmse['model'] == model]).rmse
    rmse_val = list(rmse_val)[0]
    delta = test_rmse - rmse_val
    models_delta.append(delta)

## Adding column with the modelname and attributes
validation_rmse['models_w_Attributes'] = models_w_atributes
validation_rmse['test_rmse'] = models_test_rmse
validation_rmse['delta'] = models_delta
## Picking out the model with smallest RMSE and smallest delta for generalization error

best_model = pipes[validation_rmse.loc[np.argmin(validation_rmse.delta), 'model']]

print("Best Model Validation-RMSE: ")
print(validation_rmse.loc[np.argmin(validation_rmse.delta), ['model', 'rmse', 'test_rmse', 'delta']])
validation_rmse


Best Model Validation-RMSE: 
model        most_frequent_LinearRegression_standard
rmse                                         1.30743
test_rmse                                   1.544924
delta                                       0.237494
Name: 108, dtype: object


model      rmse  \
0                     median_LinearRegression_standard  1.302203   
1                       median_LinearRegression_MinMax  1.302203   
2                       median_LinearRegression_MaxAbs  1.302203   
3                   median_LogisticRegression_standard  1.453444   
4                     median_LogisticRegression_MinMax  1.453444   
5                     median_LogisticRegression_MaxAbs  1.453444   
6                median_DecisionTreeRegressor_standard  1.501157   
7                  median_DecisionTreeRegressor_MinMax  1.558356   
8                  median_DecisionTreeRegressor_MaxAbs  1.542230   
9           median_RandomForestRegressor_n100_standard  1.321711   
10            median_RandomForestRegressor_n100_MinMax  1.303534   
11            median_RandomForestRegressor_n100_MaxAbs  1.313665   
12          median_RandomForestRegressor_n200_standard  1.302207   
13            median_RandomForestRegressor_n200_MinMax  1.307043   
14            median_RandomForestRegressor_n200_MaxAbs  1.305169   
15          median_RandomForestRegressor_n300_standard  1.303589   
16            median_RandomForestRegressor_n300_MinMax  1.302185   
17            median_RandomForestRegressor_n300_MaxAbs  1.300560   
18          median_RandomForestRegressor_n400_standard  1.310178   
19            median_RandomForestRegressor_n400_MinMax  1.300976   
20            median_RandomForestRegressor_n400_MaxAbs  1.303488   
21          median_RandomForestRegressor_n500_standard  1.309877   
22            median_RandomForestRegressor_n500_MinMax  1.300604   
23            median_RandomForestRegressor_n500_MaxAbs  1.304265   
24             median_KNeighborsRegressor_03n_standard  1.538849   
25               median_KNeighborsRegressor_03n_MinMax  1.538849   
26               median_KNeighborsRegressor_03n_MaxAbs  1.538849   
27             median_KNeighborsRegressor_05n_standard  1.338656   
28               median_KNeighborsRegressor_05n_MinMax  1.338656   
29               median_KNeighborsRegressor_05n_MaxAbs  1.338656   
30             median_KNeighborsRegressor_07n_standard  1.312227   
31               median_KNeighborsRegressor_07n_MinMax  1.312227   
32               median_KNeighborsRegressor_07n_MaxAbs  1.312227   
33             median_KNeighborsRegressor_10n_standard  1.360698   
34               median_KNeighborsRegressor_10n_MinMax  1.360698   
35               median_KNeighborsRegressor_10n_MaxAbs  1.360698   
36             median_KNeighborsRegressor_13n_standard  1.340924   
37               median_KNeighborsRegressor_13n_MinMax  1.340924   
38               median_KNeighborsRegressor_13n_MaxAbs  1.340924   
39              median_DecisionTreeClassifier_standard  1.453444   
40                median_DecisionTreeClassifier_MinMax  1.405347   
41                median_DecisionTreeClassifier_MaxAbs  1.400893   
42          median_RandomForestClassifier_250_standard  1.561249   
43            median_RandomForestClassifier_250_MinMax  1.581139   
44            median_RandomForestClassifier_250_MaxAbs  1.565248   
45          median_RandomForestClassifier_500_standard  1.577181   
46            median_RandomForestClassifier_500_MinMax  1.581139   
47            median_RandomForestClassifier_500_MaxAbs  1.577181   
48          median_RandomForestClassifier_750_standard  1.561249   
49            median_RandomForestClassifier_750_MinMax  1.592953   
50            median_RandomForestClassifier_750_MaxAbs  1.553222   
51         median_RandomForestClassifier_1000_standard  1.565248   
52           median_RandomForestClassifier_1000_MinMax  1.565248   
53           median_RandomForestClassifier_1000_MaxAbs  1.581139   
54                      mean_LinearRegression_standard  1.296255   
55                        mean_LinearRegression_MinMax  1.296255   
56                        mean_LinearRegression_MaxAbs  1.296255   
57                    mean_LogisticRegression_standard  1.462019   
58                      mean_Logistic

In [9]:

actual_y_test, predicted_y_test = Y_test, best_model.predict(X_test)

plt.figure(figsize=(14, 9))
plt.scatter(
    x = range(len(predicted_y_test)),
    y = predicted_y_test,
    label = f'Predicted Goals\nRMSE: {round(test_rmse, 4)}',
    c = 'b'
)

plt.scatter(
    x = range(len(actual_y_test)), 
    y = actual_y_test,
    label = 'Actual Goals',
    c = 'r'
)
plt.plot(
    [avg_of_train_data for i in range(len(actual_y_test))],
    c = 'black',
    label = f'Test_Data Average Goals Scored\nRMSE: {round(average_prediction_RMSE, 4)}'
)
plt.legend(bbox_to_anchor=(1.1, 1.05))
plt.ylabel("#Goals")
plt.xlabel("Prediction number")
plt.show()


### As we see, the model predicts within the accaptable range, where lower bound is 0 and uper bound in theory infinite. As well as the range holds, we see that our predicted data, being the blue scatterplots is well in our range of actual data (0-6 goals). This makes out model trustworthy. The interesting part about our prediction is that the most of the predicted goals scored is in the range between 1 and 2, wichs is because of  the average we calculated earlier, and hence makes sense.

We have a RMSE on the best model with the validation data beein ≈1.3 which is average, and corresponding test-RMSE at ≈1.54. If we compare this to the calculated RMSE from the random distribution it is clear that the model is more accurate, with a almost half the RMSE as the normal distributed prediction. Comparing it to the avergae-prediction-method, which has a RMSE of 1.46, our result is not that impressive, although it is slightly better. The test_rmse and validation_rmse makes up the delta of 0.26 which is accaptable, although it is not the best when it comes to getting a perfect generalized model.

In [10]:
## Storing our model for the website beeing created.
pickle.dump(best_model, open('app_folder/model.pkl', 'wb'))

In [11]:
## Prediction and the creation of the CSV-file with ur output from the predication.
X_test_2020 = test_games[numeric_features]
prediction = best_model.predict(X_test_2020)
prediction = [round(x) for x in prediction]
test_games['Score'] = prediction
test_games['ID'] = test_games.index
pred_cols = test_games[['ID', 'Score']]
pred_cols.to_csv("predictions.csv", index = False)
pred_cols


ID  Score
0      0      1
1      1      1
2      2      2
3      3      1
4      4      2
5      5      1
6      6      1
7      7      1
8      8      2
9      9      1
10    10      1
11    11      1
12    12      2
13    13      1
14    14      1
15    15      1
16    16      1
17    17      2
18    18      2
19    19      1
20    20      1
21    21      2
22    22      2
23    23      1
24    24      2
25    25      1
26    26      2
27    27      1
28    28      2
29    29      1
30    30      2
31    31      1
32    32      2
33    33      1
34    34      1
35    35      1
36    36      1
37    37      1
38    38      2
39    39      1
40    40      1
41    41      2
42    42      1
43    43      2
44    44      2
45    45      1
46    46      1
47    47      1
48    48      2
49    49      1
50    50      2
51    51      1
52    52      2
53    53      1
54    54      2
55    55      1
56    56      1
57    57      1
58    58      2
59    59      1
60    60      1
61    61      1
62    62      2
63    63      1
64    64      2
65    65      1
66    66      1
67    67      2
68    68      1
69    69      1
70    70      1
71    71      1
72    72      2
73    73      1
74    74      1
75    75      1
76    76      1
77    77      1
78    78      2
79    79      1
80    80      1
81    81      1
82    82      1
83    83      1
84    84      2
85    85      1
86    86      1
87    87      2
88    88      2
89    89      1
90    90      2
91    91      1
92    92      2
93    93      1
94    94      2
95    95      1
96    96      1
97    97      2
98    98      1
99    99      1
100  100      1
101  101      2
102  102      1
103  103      1
104  104      2
105  105      1
106  106      1
107  107      1
108  108      2
109  109      1
110  110      2
111  111      1
112  112      2
113  113      1
114  114      1
115  115      1
116  116      1
117  117      1
118  118      1
119  119      1
120  120      1
121  121      1
122  122      1
123  123      1
124  124      2
125  125      1
126  126      1
127  127      1
128  128      1
129  129      1
130  130      2
131  131      1
132  132      1
133  133      2
134  134      2
135  135      1
136  136      2
137  137      1
138  138      2
139  139      1
140  140      2
141  141      1
142  142      2
143  143      1
144  144      1
145  145      1
146  146      1
147  147      1
148  148      2
149  149      1
150  150      2
151  151      1
152  152      2
153  153      1
154  154      2
155  155      1
156  156      1
157  157      1
158  158      1
159  159      1
160  160      1
161  161      2
162  162      1
163  163      1
164  164      1
165  165      1
166  166      1
167  167      1
168  168      1
169  169      1
170  170      2
171  171      1
172  172      2
173  173      1
174  174      2
175  175      1
176  176      1
177  177      1
178  178      2
179  179      1
180  180      2
181  181      1
182  182      2
183  183      1